In [1]:
from web3 import Web3
import pandas as pd
import datetime
from concurrent.futures import ProcessPoolExecutor
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [9]:
#key_1 = "4d08ab7e29294be397e15ec5503401ca"
key_2 = "d4d8ddd18ef6438ea49fd3d5006b7eba"
key_3 = "b7b06ad6a7304e2197efa10b79e1c867"
key_4 = "58bce48fe80b451c96fee8ef35bbcce8"

provider = "https://mainnet.infura.io/v3/"

In [48]:
w3 = Web3(Web3.HTTPProvider(provider))

start_block = 18935888
end_block   = 18993488

transactions_data = []

for block_number in range(start_block, end_block + 1):
    block = w3.eth.get_block(block_number, full_transactions=True)
    block_timestamp = datetime.datetime.utcfromtimestamp(block.timestamp)
    
    for tx in block.transactions:
        tx_receipt = w3.eth.get_transaction_receipt(tx.hash)
        
        transactions_data.append({
            'block_number': block_number,
            'tx_hash': tx.hash.hex(),
            'gas_price': tx.gasPrice,
            'gas_used': tx_receipt.gasUsed,
            'timestamp': block_timestamp
        })

print("Dataframe saved. Number of transactions:", len(df_transactions))

df_transactions = pd.DataFrame(transactions_data)

df_transactions['gas_price_gwei'] = df_transactions['gas_price'].apply(lambda x: w3.from_wei(x, 'gwei')).astype(int)
df_transactions['tx_hash']        = df_transactions['tx_hash'].astype(str)
df_transactions['timestamp']      = pd.to_datetime(df_transactions['timestamp'])
df_transactions = df_transactions.set_index('timestamp').sort_index()

df_transactions.to_pickle('ethereum_transactions.pkl')


KeyboardInterrupt



In [ ]:
df_transactions

In [6]:
fetch_transactions([18935888], "d4d8ddd18ef6438ea49fd3d5006b7eba")

[{'block_number': 18935888,
  'tx_hash': '0x896d48c8709c9db6a69f0bfd1921792f177e4bc8fe5e06ae5e2790e80f817f46',
  'gas_price': 55770000000,
  'gas_used': 63209,
  'timestamp': datetime.datetime(2024, 1, 4, 18, 56, 11)},
 {'block_number': 18935888,
  'tx_hash': '0x8d98a2e56601e45146a1c6936be1b7a892267cbb123e73ec7fbea82871aaaf92',
  'gas_price': 34140509525,
  'gas_used': 21000,
  'timestamp': datetime.datetime(2024, 1, 4, 18, 56, 11)},
 {'block_number': 18935888,
  'tx_hash': '0x93d8fa5d49aaf50c475d70d734de3c9c11a6d627b033e364d58d68a1e6f38c60',
  'gas_price': 32000000000,
  'gas_used': 63209,
  'timestamp': datetime.datetime(2024, 1, 4, 18, 56, 11)},
 {'block_number': 18935888,
  'tx_hash': '0x46713038176eeb80348686ac0775eb9723672119919a0d112861eb4f1a8e8300',
  'gas_price': 32000000000,
  'gas_used': 59572,
  'timestamp': datetime.datetime(2024, 1, 4, 18, 56, 11)},
 {'block_number': 18935888,
  'tx_hash': '0x6713a5d964f18514b6c866132b36d8e9be09bc1a30299526f4cd4a1184b80537',
  'gas_price'

In [14]:
from web3 import Web3
import pandas as pd
from datetime import datetime
import numpy as np
from joblib import Parallel, delayed

def fetch_transactions(block_numbers, key):
    local_w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/" + key))
    transactions_data = []
    for block_number in block_numbers:
        block = local_w3.eth.get_block(block_number, full_transactions=True)
        block_timestamp = datetime.utcfromtimestamp(block.timestamp)
        for tx in block.transactions:
            tx_receipt = local_w3.eth.get_transaction_receipt(tx.hash)
            transactions_data.append({
                'block_number': block_number,
                'tx_hash': tx.hash.hex(),
                'gas_price': tx.gasPrice,
                'gas_used': tx_receipt.gasUsed,
                'timestamp': block_timestamp
            })
    return transactions_data

def main(start_block, end_block, keys):
    assert len(keys) == 3, "This setup requires exactly four API keys."
    
    num_jobs = len(keys)
    block_range = np.arange(start_block, end_block + 1)
    split_block_ranges = np.array_split(block_range, num_jobs)
    
    # Manually specify each job
    results = Parallel(n_jobs=num_jobs)([
        delayed(fetch_transactions)(split_block_ranges[0].tolist(), keys[0]),
        delayed(fetch_transactions)(split_block_ranges[1].tolist(), keys[1]),
        delayed(fetch_transactions)(split_block_ranges[2].tolist(), keys[2])
    ])
    
    # Flatten the list of lists to a single list of transactions
    all_transactions = [item for sublist in results for item in sublist]

    # Create a DataFrame from the transactions
    df_transactions = pd.DataFrame(all_transactions)
    df_transactions['gas_price_gwei'] = df_transactions['gas_price'].apply(lambda x: Web3.fromWei(x, 'gwei')).astype(int)
    df_transactions['tx_hash'] = df_transactions['tx_hash'].astype(str)
    df_transactions['timestamp'] = pd.to_datetime(df_transactions['timestamp'])
    df_transactions = df_transactions.set_index('timestamp').sort_index()

    # Save the DataFrame to a .pkl file
    df_transactions.to_pickle('ethereum_transactions.pkl')
    print(f"Dataframe saved. Number of transactions: {len(df_transactions)}")

if __name__ == '__main__':
    start_block = 18935888
    end_block   = 18993488
    provider_keys = [
        "d4d8ddd18ef6438ea49fd3d5006b7eba",
        "b7b06ad6a7304e2197efa10b79e1c867",
        "58bce48fe80b451c96fee8ef35bbcce8",
    ]
    main(start_block, end_block, provider_keys)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


HTTPError: 429 Client Error: Too Many Requests for url: https://mainnet.infura.io/v3/d4d8ddd18ef6438ea49fd3d5006b7eba

In [3]:
import logging
import os
import json
from datetime import datetime
import numpy as np
from joblib import Parallel, delayed
from web3 import Web3

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

data_dir = 'block_data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

def save_block_data(block_number, data):
    """Save block transaction data to a JSON file."""
    try:
        file_path = os.path.join(data_dir, f'{block_number}.json')
        with open(file_path, 'w') as file:
            json.dump(data, file)
        logging.info(f"Block {block_number} data saved.")
    except Exception as e:
        logging.error(f"Error saving data for block {block_number}: {e}")
        
        

def load_block_data(block_number):
    """Check if block transaction data exists as a .pkl file."""
    file_path = os.path.join(data_dir, f'{block_number}.pkl')
    if os.path.exists(file_path):
        return True
    return False


def fetch_transactions(block_numbers, key):
    local_w3 = Web3(Web3.HTTPProvider(f"https://mainnet.infura.io/v3/{key}"))
    transactions_data = []
    for block_number in block_numbers:
        block = local_w3.eth.get_block(int(block_number), full_transactions=True)
        block_timestamp = datetime.utcfromtimestamp(block.timestamp)
        for tx in block.transactions:
            tx_receipt = local_w3.eth.get_transaction_receipt(tx.hash)
            transactions_data.append({
                'block_number': block_number,
                'tx_hash'     : tx.hash.hex(),
                'gas_price'   : tx.gasPrice,
                'gas_used'    : tx_receipt.gasUsed,
                'timestamp'   : block_timestamp
            })
    return transactions_data


def process_block(block_number, key):
    """Process a single block, checking for existing data and fetching if not present."""
    if load_block_data(block_number):
        logging.info(f"Data for block {block_number} already exists in .pkl format. Skipping fetch.")
        return

    block_data = fetch_transactions(block_number, key)
    if block_data:
        save_block_data(block_number, block_data)
        
        

def main(start_block, end_block, keys):
    """Main function to process blocks in a given range using multiple keys."""
    logging.info("Starting processing blocks.")
    
    block_range = np.arange(start_block, end_block + 1)
    num_jobs = len(keys)
    split_block_ranges = np.array_split(block_range, num_jobs)

    results = Parallel(n_jobs=num_jobs)(delayed(process_block)(block, keys[i % len(keys)])
                                    for i, block_range in enumerate(split_block_ranges)
                                    for block in block_range)


    logging.info(f"Processing complete. Processed {len(results)} blocks.")

start_block = 18935888
end_block   = 18935890
keys = ["d4d8ddd18ef6438ea49fd3d5006b7eba",
        "b7b06ad6a7304e2197efa10b79e1c867",
        "58bce48fe80b451c96fee8ef35bbcce8",]

main(start_block, end_block, keys)


2024-02-07 14:25:27,531 - INFO - Starting processing blocks.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


TypeError: 'numpy.int64' object is not iterable

In [5]:
from web3 import Web3

key = "d4d8ddd18ef6438ea49fd3d5006b7eba"  # Replace with your actual project ID
local_w3 = Web3(Web3.HTTPProvider(f"https://mainnet.infura.io/v3/{key}"))

try:
    block = local_w3.eth.get_block('latest')  # Fetches the latest block
    print(block)
except Exception as e:
    print(f"Error fetching latest block: {e}")


AttributeDict({'baseFeePerGas': 53387181796, 'difficulty': 0, 'extraData': HexBytes('0x407273796e636275696c646572'), 'gasLimit': 30000000, 'gasUsed': 15272564, 'hash': HexBytes('0xb1cd5f3ac83e1bd3fc7bfc53fbd05b7e7da35b73154483c3996767dad5b54581'), 'logsBloom': HexBytes('0x6273c16bc1b0134082384010f89192c14f4ee829049b210ec50927fed4ea4c41002cb44d090e72bdb3203910358c6f2182416b2d8c3475f22104a52a5dee2c584637406f55331d2d7d5ae8dc00ec74b1853a10811a465c810a0618d0a12481029253f42c1b137b0200c4599540caef95602d4843c61857666e610096844c2107a3301df0824a001630702169018403fd043000216564d6da4c06cc530434ba5d4a10892c1b00f0062a4414e4d00a951a05946990213e80308c649cae6c18227172d365eac5580442210a9331410810c4074f8306f48d70d91905138a5836bee00e73280b168549200105f917084002a9917c924b9228054501bcf81a7a05d449'), 'miner': '0x1f9090aaE28b8a3dCeaDf281B0F12828e676c326', 'mixHash': HexBytes('0xf9bf6db68f17895e18a7ec03420e3b9b0ef704adcdaaffc319f2e823a69586c2'), 'nonce': HexBytes('0x0000000000000000'), 'number': 19178798, 'par